In [185]:
# Загружаем необходимые библиотеки

from scipy.io import wavfile
from scipy.fftpack import fft
import matplotlib.pyplot as plt
#from matplotlib.pyplot import figure
from IPython.display import Audio
import numpy as np
import random

%matplotlib inline

In [174]:
sample_rate = 44100

sig = []

t0 = 0

delta = 1/sample_rate

coef = 100

string = hem_code
          
for i in range(len(string)):
    c = string[i]
    if c == '0':
        delta = 1/sample_rate
    if c == '1':
        delta = 2/sample_rate

        
    for j in range(sample_rate//coef):
        t0 = t0 + delta
        sig.append(5*np.sin(2*np.pi*440*t0))

In [152]:
Audio(data=sig,rate=sample_rate)

In [175]:
sample_rate, data = wavfile.read('/Users/mihail/Record29.wav')

In [176]:
sample_rate

48000

In [177]:
data = data[:,0]

In [178]:
Audio(data=data,rate=sample_rate)

In [252]:
A = []

move = 0 
start = 0
text = '' 
for i in range(len(data)//(sample_rate//coef)):
    temp = data[sample_rate//coef*i-move:sample_rate//coef*(i+1)-move]
    f = fft(temp,sample_rate)
    res = np.argmax(np.abs(f)[0:2000])
    
    #if np.sum(temp**2) < 22618611114:
    #    continue
    A.append(sum(temp**2))
    
    if res > 400 and res < 480:
        if start == 0:
            start = 1
            move = len(temp) - np.argmax(temp)
        else:
            text += '0'
        
    elif res > 800 and res < 960:
        if start == 0:
            start = 1
            move = len(temp) - np.argmax(temp)
        else:
            text += '1'
    elif start == 1:
        text += '0'   
text = text[:len(hem_code)] 
print(text)


01101011000011001110101110100001111101110000110101000011011101110110001000001111010001010001110111101010000001110100010100001001010111010000101000001111101010100001101110010111010000110100100001010010000001110100001101111101111011010000011011000011001001000001101010101111010000111100110000101110100001101100001110110010000110011101101101000111101010000010111010001010001110111101010000001110100001101101010000111010000011011010111101110100001101110011011010000110100110001011101010010100000100110001100101010000110010010000011001101010010100101101011010001010100001001001100100001101110000000001000000110101000011011101001101100100001110110000111010001010110011000100110100010100010001110000100001100100001000000


In [227]:
string = '11010000 10111010 11010000 10110000 11010000 10111010 00100000 11010001 10001111 00100000 11010001 10000001 11010001 10000011 11010000 10111010 11010000 10110000 00100000 11010000 10110111 11010000 10110000 11010000 10110101 11010000 10110001 11010000 10110000 11010000 10111011 11010001 10000001 11010001 10001111 00100000 11010000 10110100 11010000 10110101 11010000 10111011 11010000 10110000 11010001 10000010 11010001 10001100 00100000 11010001 10001101 11010001 10000010 11010000 10111000 00100000 11010000 10111011 11010000 10110000 11010000 10110001 11010001 10001011 00100001'

In [232]:
block = string.split(" ")

In [233]:
len(block)

65

In [73]:
Check_sum = ''
for j in range (8):
    cur_sum = 0
    for i in range (len(block)):
        cur_sum = (cur_sum+int(block[i][j]))%2
    Check_sum += str(cur_sum)

In [74]:
block.append(Check_sum)

In [75]:
new_code = ''
for j in range(len(block)):
    cur_sum = 0
    for i in range (8):
        cur_sum = (cur_sum+int(block[j][i]))%2
    new_code += block[j] + str(cur_sum)

In [79]:
new_code

'1101000011011101011101000011011000011101000011011101010010000011101000101000111110010000011101000101000000101101000101000001111101000011011101011101000011011000010010000011101000011011011101101000011011000011101000011011010111101000011011000101101000011011000011101000011011101101101000101000000101101000101000111110010000011101000011011010001101000011011010111101000011011101101101000011011000011101000101000001001101000101000110010010000011101000101000110101101000101000001001101000011011100000010000011101000011011101101101000011011000011101000011011000101101000101000101100010000100100001000000'

In [78]:
while (len(new_code)%26!=0):
    new_code+= "0"

In [80]:
hem_blocks = []
while (len(new_code)!=0):
    hem_blocks.append(new_code[:26])
    new_code=new_code[26:]

In [84]:
hem_blocks[1]

'11011000011101000011011101'

In [82]:
def encode_hemming(string):
    lst = [0,0]
        
    bit_count = 3
    
    for i in string:
        if np.log2(bit_count)%1 == 0:
            lst.append(0)
            bit_count +=1 
            
        lst.append(int(i))
        bit_count +=1

    for i in range(len(lst)):
        if np.log2(i+1)%1 == 0:

            check_sum = 0
            for j in range(i,len(lst),2**int((np.log2(i+1)+1))):
                check_sum += sum(lst[j:int(j+2**(np.log2(i+1)))])
                
            check_sum %= 2
            lst[i] = check_sum
           
    result = ''
    
    for i in lst:
        result += str(i)
    return result

In [83]:
def decode_hemming(string):
    lst = [int(i) for i in string]
    
    old_checksums = []
    
    for i in range(len(lst)):
        if np.log2(i+1)%1 == 0:
            old_checksums.append(lst[i])
            lst[i] = 0
    
    new_checksums = []
    for i in range(len(lst)):
        if np.log2(i+1)%1 == 0:
            check_sum = 0
            for j in range(i,len(lst),2**int((np.log2(i+1)+1))):
                check_sum += sum(lst[j:int(j+2**(np.log2(i+1)))])
            check_sum %= 2
            new_checksums.append(check_sum)
    
    res = np.array(old_checksums) != np.array(new_checksums)
    
    s = 0
    for i in range(len(res)):
        s+=res[i]*(2**i)
        
    decoded_string = ''

    for i in range(len(string)):
        if np.log2(i+1)%1 == 0:
            continue
        if i == s-1:
            if string[i] == '0':
                decoded_string += '1'
            else:
                decoded_string += '0'
        else:
            decoded_string += string[i]
            
    return decoded_string

In [85]:
hem_code = ''
for hem_block in hem_blocks:
    hem_code+= encode_hemming(hem_block)

In [87]:
len(hem_code)

713

In [195]:
len(text)

713

In [193]:
count = 0
for i in range(len(hem_code)):
    if hem_code[i] != text[i]:
        print(i)
        count +=1

51
83
88
223
251
293
383
438
600
620
641
689


In [194]:
count

12

In [192]:
#так как запись вышла очень удачной то просто внесём 20 ошибок в наш код
for i in range (20):
    pos = random.randint(0,len(hem_code)-1)
    text =text[:pos-1] + str((int(text[pos])+1)%2) + text[pos:]

In [196]:
enc_blocks= []
while(len(text)!=0):
    enc_blocks.append(text[:31])
    text=text[31:]

In [198]:
nohem_code=''
for enc_block in enc_blocks:
    nohem_code+=decode_hemming(enc_block)
    

In [199]:
nohem_code

'1101000011011101011101000011011000011101000011011101010010001011101010101100111110010000011101000101000000101101000101000001111101000011011101011101000011011000010010000011101000011011011101101000011011000011101000011011010111101000011011000101101000011011000011101000011011101101101000101000000101101000101000111110010000011101000011011010001101000011011010111101000011011101101101000011011000011101000101000001001101000101000110010010000011101000101000110101101000101000001001101000011011100000010000011101000011011101101101000011011011011101000011011000101101000101000101100010000100100001000000'

In [200]:
checker = '1101000011011101011101000011011000011101000011011101010010000011101000101000111110010000011101000101000000101101000101000001111101000011011101011101000011011000010010000011101000011011011101101000011011000011101000011011010111101000011011000101101000011011000011101000011011101101101000101000000101101000101000111110010000011101000011011010001101000011011010111101000011011101101101000011011000011101000101000001001101000101000110010010000011101000101000110101101000101000001001101000011011100000010000011101000011011101101101000011011000011101000011011000101101000101000101100010000100100001000000'

In [201]:
len(checker)

598

In [202]:
len(nohem_code)

598

In [203]:
count = 0
for i in range(len(checker)):
    if nohem_code[i] != checker[i]:
        print(i)
        count +=1

60
68
73
536
537


In [204]:
final_blocks = []
while(len(nohem_code)!=0):
    final_blocks.append(nohem_code[:9])
    nohem_code=nohem_code[9:]

In [211]:
len(final_blocks)

66

In [210]:
if len(final_blocks[len(final_blocks)-1])!=9:
    del(final_blocks[len(final_blocks)-1])

In [219]:
check_list1=[]
for i in range(8):
    cur_sum = 0
    for j in range(len(final_blocks)-1):
        cur_sum = (cur_sum + int(final_blocks[j][i]))%2
    if (cur_sum != int(final_blocks[len(final_blocks)-1][i])):
        check_list1.append(i)
    

In [220]:
check_list1

[1]

In [221]:
check_list2 = []
for i in range(len(final_blocks)):
    cur_sum = 0
    for j in range(8):
        cur_sum = (cur_sum + int(final_blocks[i][j]))%2
    if (cur_sum != int(final_blocks[i][8])):
        check_list2.append(i)

In [222]:
check_list2

[6, 7, 8]

In [247]:
for i in check_list2:
    for j in check_list1:
        final_blocks[i]=final_blocks[i][:j-1] + str((int(final_blocks[i][j])+1)%2) + final_blocks[i][j:]

In [238]:
del(final_blocks[len(final_blocks)-1])

In [248]:
vivod = ''
for block in final_blocks:
    vivod += block[:8]+ " "
    

In [249]:
len(vivod)

585

In [250]:
len(string)

584

In [251]:
for i in range(len(string)):
    if vivod[i] != string[i]:
        print(i)

54
60
63
68
72
73
536
537
